# BentoML Demo: Titanic Survival Prediction with XGBoost


BentoML is an open-source framework for machine learning **model serving**, aiming to **bridge the gap between Data Science and DevOps.**

Data Scientists can easily package their models trained with any ML framework using BentoMl and reproduce the model for serving in production. BentoML helps with managing packaged models in the BentoML format, and allows DevOps to deploy them as online API serving endpoints or offline batch inference jobs, on any cloud platform.

Before reading this example project, be sure to check out the [Getting started guide](https://github.com/bentoml/BentoML/blob/master/guides/quick-start/bentoml-quick-start-guide.ipynb) to learn about the basic concepts in BentoML.


This notebook demonstrates use BentoML to serve a model trained with the XGBoost framework, specifically using the Titanic Survival dataset.


Let's get started!
![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=xgboost&ea=xgboost-tiantic-survival-prediction&dt=xgboost-tiantic-survival-prediction)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install -q --upgrade xgboost==0.90 numpy==1.18.5 pandas==1.0.4 bentoml

In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
import bentoml

# Prepare Dataset
download dataset from https://www.kaggle.com/c/titanic/data

In [4]:
!mkdir data
!curl https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/train.csv -o ./data/train.csv
!curl https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/test.csv -o ./data/test.csv

mkdir: data: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 60302  100 60302    0     0   180k      0 --:--:-- --:--:-- --:--:--  180k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 28210  100 28210    0     0  99330      0 --:--:-- --:--:-- --:--:-- 99330


In [5]:
train = pd.read_csv("./data/train.csv")
test  = pd.read_csv("./data/test.csv")
X_y_train = xgb.DMatrix(data=train[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']], label= train['Survived'])
X_test    = xgb.DMatrix(data=test[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']])

In [6]:
train[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch', 'Survived']].head()

,Pclass,Age,Fare,SibSp,Parch,Survived
0,3,22.0,7.2500,1,0,0
1,1,38.0,71.2833,1,0,1
2,3,26.0,7.9250,0,0,1
3,1,35.0,53.1000,1,0,1
4,3,35.0,8.0500,0,0,0


# Model Training

In [7]:
params = {
          'base_score': np.mean(train['Survived']),
          'eta':  0.1,
          'max_depth': 3,
          'gamma' :3,
          'objective'   :'reg:linear',
          'eval_metric' :'mae'
         }
model = xgb.train(params=params, 
                  dtrain=X_y_train, 
                  num_boost_round=3)

[17:27:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [8]:
y_test =  model.predict(X_test)
test['pred'] = y_test
test[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch','pred']].iloc[10:].head(2)

,Pclass,Age,Fare,SibSp,Parch,pred
10,3,NaN,7.8958,0,0,0.341580
11,1,46.0,26.0000,0,0,0.413966


## Create BentoService for model serving

In [9]:
%%writefile xgboost_titanic_bento_service.py

import xgboost as xgb

import bentoml
from bentoml.frameworks.xgboost import XgboostModelArtifact
from bentoml.adapters import DataframeInput

@bentoml.env(auto_pip_dependencies=True)
@bentoml.artifacts([XgboostModelArtifact('model')])
class TitanicSurvivalPredictionXgBoost(bentoml.BentoService):
    
    @bentoml.api(input=DataframeInput())
    def predict(self, df):
        data = xgb.DMatrix(data=df[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']])
        return self.artifacts.model.predict(data)

Overwriting xgboost_titanic_bento_service.py


# Create BentoService saved bundle

In [10]:
# 1) import the custom BentoService defined above
from xgboost_titanic_bento_service import TitanicSurvivalPredictionXgBoost

# 2) `pack` it with required artifacts
bento_service = TitanicSurvivalPredictionXgBoost()
bento_service.pack('model', model)

# 3) save your BentoSerivce
saved_path = bento_service.save()

[2020-06-16 17:27:43,973] INFO - BentoService bundle 'TitanicSurvivalPredictionXgBoost:20200616172725_4CD6B0' saved to: /Users/chaoyu/bentoml/repository/TitanicSurvivalPredictionXgBoost/20200616172725_4CD6B0


## REST API Model Serving


To start a REST API model server with the BentoService saved above, use the bentoml serve command:

In [15]:
!bentoml serve TitanicSurvivalPredictionXgBoost:latest

[2020-06-16 17:27:58,816] INFO - Getting latest version TitanicSurvivalPredictionXgBoost:20200616172725_4CD6B0
[17:27:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
 * Serving Flask app "TitanicSurvivalPredictionXgBoost" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Jun/2020 17:28:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2020 17:28:01] "GET /docs.json HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2020 17:28:14] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2020 17:28:17] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2020 17:28:17] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2020 17:28:17] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2020 17:28:17] "POST /predict HTTP/1.1" 200 -
^C


If you are running this notebook from Google Colab, you can start the dev server with `--run-with-ngrok` option, to gain acccess to the API endpoint via a public endpoint managed by [ngrok](https://ngrok.com/):

In [ ]:
!bentoml serve TitanicSurvivalPredictionXgBoost:latest --run-with-ngrok

Open http://127.0.0.1:5000 to see more information about the REST APIs server in your
browser.


### Send prediction requeset to the REST API server

Run the following `curl` command to send request data to REST API server and get a prediction result:

```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '[{"Pclass": 1, "Age": 30, "Fare": 200, "SibSp": 1, "Parch": 0}]' \
localhost:5000/predict
```

## Containerize model server with Docker


One common way of distributing this model API server for production deployment, is via Docker containers. And BentoML provides a convenient way to do that.

Note that docker is **not available in Google Colab**. You will need to download and run this notebook locally to try out this containerization with docker feature.

If you already have docker configured, simply run the follow command to product a docker container serving the IrisClassifier prediction service created above:

In [16]:
!bentoml containerize TitanicSurvivalPredictionXgBoost:latest

sha256:986cbfc72fbca0645a54267df982f4970df5fb408d388fd9f2c7502084d7a686


Next, you can docker push the image to your choice of registry for deployment, or run it locally for development and testing:

In [17]:
!docker run -p 5000:5000 titanic-survival-predict-server

[2020-06-17 00:29:53,084] INFO - get_gunicorn_num_of_workers: 3, calculated by cpu count
[2020-06-17 00:29:53 +0000] [1] [INFO] Starting gunicorn 20.0.4
[2020-06-17 00:29:53 +0000] [1] [INFO] Listening at: http://0.0.0.0:5000 (1)
[2020-06-17 00:29:53 +0000] [1] [INFO] Using worker: sync
[2020-06-17 00:29:53 +0000] [12] [INFO] Booting worker with pid: 12
[2020-06-17 00:29:53 +0000] [13] [INFO] Booting worker with pid: 13
[2020-06-17 00:29:53 +0000] [14] [INFO] Booting worker with pid: 14
^C
[2020-06-17 00:34:02 +0000] [1] [INFO] Handling signal: int
[2020-06-17 00:34:02 +0000] [14] [INFO] Worker exiting (pid: 14)
[2020-06-17 00:34:02 +0000] [13] [INFO] Worker exiting (pid: 13)
[2020-06-17 00:34:02 +0000] [12] [INFO] Worker exiting (pid: 12)
[00:29:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:29:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederr

## Load saved BentoService

bentoml.load is the API for loading a BentoML packaged model in python:

In [11]:
import bentoml

loaded_svc = bentoml.load(saved_path)

result = loaded_svc.predict(test)
test['pred'] = result
test[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch','pred']]

[2020-06-16 17:27:44,282] WARNING - Module `xgboost_titanic_bento_service` already loaded, using existing imported module.
[17:27:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


,Pclass,Age,Fare,SibSp,Parch,pred
0,3,34.5,7.8292,0,0,0.341580
1,3,47.0,7.0000,1,0,0.341580
2,2,62.0,9.6875,0,0,0.371730
3,3,27.0,8.6625,0,0,0.341580
4,3,22.0,12.2875,1,1,0.341580
...,...,...,...,...,...,...
413,3,NaN,8.0500,0,0,0.341580
414,1,39.0,108.9000,0,0,0.469721
415,3,38.5,7.2500,0,0,0.341580
416,3,NaN,8.0500,0,0,0.341580


## Launch inference job from CLI

BentoML cli supports loading and running a packaged model from CLI. With the DataframeInput adapter, the CLI command supports reading input Dataframe data from CLI argument or local csv or json files:

In [14]:
!bentoml run TitanicSurvivalPredictionXgBoost:latest predict \
    --input '[{"Pclass": 1, "Age": 30, "Fare": 200, "SibSp": 1, "Parch": 0}]' 

[2020-06-16 17:27:50,818] INFO - Getting latest version TitanicSurvivalPredictionXgBoost:20200616172725_4CD6B0
[17:27:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0.46972126]


# Deployment Options

If you are at a small team with limited engineering or DevOps resources, try out automated deployment with BentoML CLI, currently supporting AWS Lambda, AWS SageMaker, and Azure Functions:
- [AWS Lambda Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_lambda.html)
- [AWS SageMaker Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_sagemaker.html)
- [Azure Functions Deployment Guide](https://docs.bentoml.org/en/latest/deployment/azure_functions.html)

If the cloud platform you are working with is not on the list above, try out these step-by-step guide on manually deploying BentoML packaged model to cloud platforms:
- [AWS ECS Deployment](https://docs.bentoml.org/en/latest/deployment/aws_ecs.html)
- [Google Cloud Run Deployment](https://docs.bentoml.org/en/latest/deployment/google_cloud_run.html)
- [Azure container instance Deployment](https://docs.bentoml.org/en/latest/deployment/azure_container_instance.html)
- [Heroku Deployment](https://docs.bentoml.org/en/latest/deployment/heroku.html)

Lastly, if you have a DevOps or ML Engineering team who's operating a Kubernetes or OpenShift cluster, use the following guides as references for implementating your deployment strategy:
- [Kubernetes Deployment](https://docs.bentoml.org/en/latest/deployment/kubernetes.html)
- [Knative Deployment](https://docs.bentoml.org/en/latest/deployment/knative.html)
- [Kubeflow Deployment](https://docs.bentoml.org/en/latest/deployment/kubeflow.html)
- [KFServing Deployment](https://docs.bentoml.org/en/latest/deployment/kfserving.html)
- [Clipper.ai Deployment Guide](https://docs.bentoml.org/en/latest/deployment/clipper.html)

